# ML/DL Mathematik – kompakter Spickzettel

KaTeX/LaTeX-Formeln für zentrale Konzepte mit kurzen Demos.


## Lineare Regression & MSE


Loss: $$\mathcal{L}(w) = \frac{1}{N}\sum_{i=1}^N (y_i - \hat y_i)^2, \quad \hat y = Xw$$

Gradient: $$\nabla_w \mathcal{L} = -\frac{2}{N} X^\top (y - Xw)$$

Closed Form: $$w = (X^\top X)^{-1} X^\top y$$

Regularisierung (Ridge): $$\mathcal{L}_{ridge} = \text{MSE} + \lambda \|w\|_2^2$$



### Erklärungen zu den Formeln:

- **Loss (MSE):** Misst, wie weit die Vorhersagen $$\hat y$$ von den echten Werten $$y$$ entfernt sind. Je kleiner, desto besser passt das Modell.
- **Gradient:** Zeigt, wie man die Gewichte $$w$$ ändern muss, um den Fehler zu verringern. Wird für das Training per "Gradient Descent" genutzt.
- **Closed Form:** Die optimale Lösung für $$w$$, wenn die Daten und das Modell einfach genug sind. Man muss nicht iterativ lernen.
- **Ridge-Regularisierung:** Verhindert, dass die Gewichte zu groß werden (Überanpassung). $$\lambda$$ steuert die Stärke der Bestrafung.

In [5]:
import numpy as np
np.random.seed(0)
X = np.random.randn(200, 1)
y = 3 * X[:,0] + 0.5 + 0.3 * np.random.randn(200)
Xb = np.c_[np.ones_like(X), X]

w = np.zeros(2)
alpha = 0.1
for _ in range(200):
    y_hat = Xb @ w
    grad = -(2/len(Xb)) * Xb.T @ (y - y_hat)
    w -= alpha * grad
w


array([0.46011025, 3.0209476 ])

### Demo: Gradient Descent für Lineare Regression

In diesem Beispiel werden die Gewichte $$w$$ mit Gradient Descent gelernt. Das Ziel: Die Gerade soll möglichst gut zu den Daten passen.

- **X**: Eingabewerte (Merkmale)
- **y**: Zielwerte (Labels)
- **w**: Gewichte der Geraden
- **alpha**: Lernrate (wie groß die Schritte sind)
- **grad**: Richtung, in die $$w$$ angepasst werden muss

Nach 200 Schritten ist $$w$$ so gewählt, dass der Fehler möglichst klein ist.

## Logistische Regression & Cross-Entropy


Sigmoid: $$\sigma(z) = \frac{1}{1 + e^{-z}}$$

Vorhersage: $$\hat y = \sigma(Xw)$$

Loss: $$\mathcal{L} = -\frac{1}{N}\sum_i \big[y_i \log \hat y_i + (1-y_i) \log(1-\hat y_i)\big]$$

Gradient: $$\nabla_w \mathcal{L} = \frac{1}{N} X^\top (\hat y - y)$$



### Erklärungen zu den Formeln:

- **Sigmoid:** Wandelt beliebige Zahlen in Werte zwischen 0 und 1 um – ideal für Wahrscheinlichkeiten.
- **Vorhersage:** Das Modell sagt die Wahrscheinlichkeit für Klasse 1 voraus.
- **Loss (Cross-Entropy):** Bestraft falsche Vorhersagen besonders stark. Je kleiner, desto besser.
- **Gradient:** Zeigt, wie die Gewichte $$w$$ angepasst werden müssen, damit die Vorhersagen besser werden.

## Optimierung


- Gradient Descent: $$w_{t+1} = w_t - \eta \nabla_w \mathcal{L}$$

- Momentum: $$v_{t+1} = \beta v_t + (1-\beta) \nabla_w \mathcal{L}, \quad w_{t+1} = w_t - \eta v_{t+1}$$

- Adam (vereinfacht): $$m_t=\beta_1 m_{t-1} + (1-\beta_1)g_t, \ v_t=\beta_2 v_{t-1} + (1-\beta_2)g_t^2, \ w_{t+1} = w_t - \eta \frac{m_t}{\sqrt{v_t}+\epsilon}$$



### Erklärungen zu den Formeln:

- **Gradient Descent:** Das Modell lernt, indem es die Gewichte Schritt für Schritt in Richtung des geringeren Fehlers anpasst.
- **Momentum:** Sorgt dafür, dass das Modell nicht in kleinen Tälern stecken bleibt und schneller lernt.
- **Adam:** Ein fortschrittlicher Algorithmus, der das Lernen noch robuster und schneller macht, indem er verschiedene Lernraten für jede Richtung nutzt.

## Regularisierung

- L2 (Ridge): $$\lambda \|w\|_2^2$$
- L1 (Lasso): $$\lambda \|w\|_1$$
- Dropout: zufälliges Nullsetzen von Neuronen im Training.
- Early Stopping: Stop bei steigendem Val-Loss.


### Erklärungen zu den Formeln:

- **L2 (Ridge):** Bestraft große Gewichte, damit das Modell nicht zu stark auf einzelne Datenpunkte reagiert.
- **L1 (Lasso):** Sorgt dafür, dass manche Gewichte ganz auf Null gesetzt werden – das Modell wird sparsamer.
- **Dropout:** Hilft, Überanpassung zu vermeiden, indem zufällig Teile des Modells beim Training ignoriert werden.
- **Early Stopping:** Stoppt das Training, bevor das Modell anfängt, sich zu sehr an die Trainingsdaten zu klammern.

## Entscheidungsbäume & Ensemble-Signale


- Entropie: $$H = -\sum_k p_k \log p_k$$

- Gini: $$G = 1 - \sum_k p_k^2$$

- Informationsgewinn: $$IG = H(\text{parent}) - \sum_j \frac{N_j}{N} H(j)$$

- Bagging/Boosting: viele schwache Modelle, gemittelt/gewichtet.


### Erklärungen zu den Formeln:

- **Entropie:** Misst, wie "gemischt" die Klassen sind. Je höher, desto unsicherer die Entscheidung.
- **Gini:** Alternative zu Entropie, misst ebenfalls die Reinheit eines Knotens im Baum.
- **Informationsgewinn:** Zeigt, wie viel besser ein Split die Daten trennt. Je höher, desto besser.
- **Bagging/Boosting:** Viele einfache Modelle werden kombiniert, um ein starkes Modell zu bauen.

## Bias-Variance

Bias = systematischer Fehler (Unteranpassung), Varianz = Sensitivität auf Rauschen (Überanpassung). Regularisierung/mehr Daten/Ensembles balancieren Bias/Varianz.


### Erklärungen zu den Formeln:

- **Bias:** Das Modell ist zu einfach und verpasst wichtige Muster (Unteranpassung).
- **Varianz:** Das Modell ist zu komplex und reagiert zu stark auf Zufall (Überanpassung).
- **Lösung:** Mit Regularisierung, mehr Daten oder Ensemble-Methoden kann man Bias und Varianz ausbalancieren.

## Deep Learning Basics


Dense-Forward: $$h = f(Wx + b)$$

Backprop (vereinfacht): $$\frac{\partial L}{\partial W} = \delta x^T, \ \delta = \frac{\partial L}{\partial h} \cdot f'(z)$$

Softmax: $$\text{softmax}(z_i) = \frac{e^{z_i}}{\sum_j e^{z_j}}$$

Init: Xavier/Glorot (tanh/sigmoid), He (ReLU/GELU). BatchNorm: $$\hat x = \frac{x-\mu}{\sqrt{\sigma^2+\epsilon}}, \ y = \gamma \hat x + \beta$$



### Erklärungen zu den Formeln:

- **Dense-Forward:** Ein Neuron berechnet eine gewichtete Summe der Eingaben und wendet eine Aktivierungsfunktion an.
- **Backpropagation:** Zeigt, wie die Gewichte angepasst werden, damit der Fehler kleiner wird.
- **Softmax:** Wandelt die Ausgaben in Wahrscheinlichkeiten um, sodass sie zusammen 1 ergeben.
- **BatchNorm:** Normalisiert die Eingaben, damit das Training stabiler und schneller wird.
- **Init:** Die Startwerte der Gewichte sind wichtig, damit das Modell gut lernen kann.

## CNNs: Shapes


Output: $$\text{out} = \left\lfloor \frac{n + 2p - k}{s} \right\rfloor + 1$$ (Kernel k, Stride s, Padding p). Pooling analog. Transfer Learning: Pretrained Backbone + kleiner Head.


### Erklärungen zu den Formeln:

- **CNN Output-Shape:** Zeigt, wie groß das Ergebnis nach einer Faltung ist. Wichtig für die Architektur.
- **Pooling:** Reduziert die Größe und hebt wichtige Merkmale hervor.
- **Transfer Learning:** Man nutzt ein vortrainiertes Modell und passt nur den letzten Teil für die eigene Aufgabe an.

## Attention


$$\text{Att}(Q,K,V) = \text{softmax}\left(\frac{QK^\top}{\sqrt{d_k}}\right) V$$

Multi-Head: mehrere Projektionen von Q/K/V, concat, linear.


### Erklärungen zu den Formeln:

- **Attention:** Das Modell lernt, auf die wichtigsten Teile der Eingabe zu achten. Die Formel zeigt, wie die Aufmerksamkeit berechnet wird.
- **Multi-Head:** Mehrere "Blicke" auf die Daten gleichzeitig, damit das Modell verschiedene Muster erkennt.

## Metriken

- Klassifikation: Accuracy, Precision/Recall/F1, ROC-AUC, PR-AUC, Confusion Matrix.
- Regression: MAE, MSE/RMSE, R².
- Calibration: ECE/Brier.
- Monitoring: Drift (Feature/Prediction), Latenz, Fehlerraten.


### Erklärungen zu den Formeln:

- **Klassifikation:** Misst, wie gut das Modell zwischen Klassen unterscheidet (z.B. Katzen vs. Hunde).
- **Regression:** Misst, wie nah die Vorhersagen an den echten Zahlen liegen.
- **Calibration:** Zeigt, ob die Wahrscheinlichkeiten des Modells wirklich stimmen.
- **Monitoring:** Überwacht, ob sich die Daten oder das Modell im Laufe der Zeit verändern.

In [6]:
import numpy as np
z = np.array([2.0, 1.0, 0.1])
softmax = np.exp(z) / np.exp(z).sum()
softmax


array([0.65900114, 0.24243297, 0.09856589])

### Demo: Softmax für Wahrscheinlichkeiten

Hier wird gezeigt, wie die Softmax-Funktion aus beliebigen Zahlen (z.B. Modell-Ausgaben) Wahrscheinlichkeiten macht:

- **z**: Die Roh-Ausgaben des Modells (z.B. für drei Klassen)
- **softmax**: Die Wahrscheinlichkeiten für jede Klasse, immer zwischen 0 und 1 und zusammen genau 1

Das ist wichtig für Klassifikation, z.B. bei Bildern: Das Modell sagt, wie wahrscheinlich jede Klasse ist.